In [5]:
import numpy as np
import pandas as pd
import tensorflow as tf
import keras
import pathlib
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [1]:
def yolo2coco(image_width, bboxes):
    """
    convert yolo annottion to coco annottion for easier usage with keras
    yolo => [xmid, ymid, w, h] (normalized)
    coco => [xmin, ymin, w, h]

    """
    # conversion (xmid, ymid) => (xmin, ymin)
    bboxes[0] = bboxes[0] - bboxes[2] * 0.5
    bboxes[1] = bboxes[1] - bboxes[3] * 0.5

    # denormalizing
    bboxes = [int(value * image_width) for value in bboxes]

    return bboxes


train_label_directory = pathlib.Path(
    r"C:\Users\loren\Documents\AI_BME\traffic\archive\TrafficSignLocalizationandDetection\train\labels"
)
val_label_directory = pathlib.Path(
    r"C:\Users\loren\Documents\AI_BME\traffic\archive\TrafficSignLocalizationandDetection\valid\labels"
)

train_labelpaths = train_label_directory.glob("*.txt")
val_lablepaths = val_label_directory.glob("*.txt")


# load all labels from disk into a list
def get_labels(labelpaths):
    labels = []
    for path in labelpaths:
        with open(path) as txt_file:
            label = txt_file.readlines()
            label = label[0].split(" ")
            label = [float(x) for x in label]
            label = yolo2coco(416, label[1:])
            labels.append(label)
    return labels


train_labels = get_labels(train_labelpaths)
val_labels = get_labels(val_lablepaths)

NameError: name 'pathlib' is not defined

In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import os

# Define the paths to your image directory and annotations
train_directory = "C:/Users/loren\Documents/AI_BME/traffic/archive/TrafficSignLocalizationandDetection/train/images"
val_directory = "C:/Users/loren\Documents/AI_BME/traffic/archive/TrafficSignLocalizationandDetection/valid/images"

# Load and preprocess the images
datagen = ImageDataGenerator(rescale=1.0 / 255)  # You can customize this

# Define a generator for images and labels
train_generator = datagen.flow_from_directory(
    train_directory,
    class_mode=None,  # No class labels, we'll use bounding box annotations
    target_size=(416, 416),  # Specify image dimensions
    batch_size=1,
)

val_generator = datagen.flow_from_directory(
    val_directory,
    class_mode=None,  # No class labels, we'll use bounding box annotations
    target_size=(416, 416),  # Specify image dimensions
    batch_size=1,
)


# Create a list of all images
def create_dataset(image_generator):
    images = []
    for i in range(len(image_generator)):
        image = np.array(image_generator[i].reshape(416, 416, 3))
        images.append((image))
    return images


# Create the dataset
train_images = create_dataset(train_generator)
val_images = create_dataset(val_generator)

Found 1392 images belonging to 1 classes.
Found 173 images belonging to 1 classes.


In [4]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import MeanSquaredError
from tensorflow.keras.metrics import MeanIoU


# Define your SSD model
def create_ssd_model(input_shape, num_classes):
    base_model = VGG16(weights="imagenet", include_top=False, input_shape=input_shape)
    x = base_model.layers[-5].output
    x = layers.Conv2D(1024, (3, 3), activation="relu", padding="same")(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation="relu")(x)
    predictions = layers.Dense(num_classes, activation="sigmoid")(x)

    model = models.Model(inputs=base_model.input, outputs=predictions)
    return model


# Create the SSD model
input_shape = (416, 416, 3)  # Adjust to your image size
num_classes = 4  # Modify to the number of classes or bounding box parameters
ssd_model = create_ssd_model(input_shape, num_classes)

# Compile the model
ssd_model.compile(
    optimizer=Adam(learning_rate=0.001), loss=MeanSquaredError(), metrics=["mae", "mse"]
)

# Train the model
# Define your training data and labels, and use model.fit() to train the model
# Make sure your training data includes bounding box coordinates as labels
# You might need to preprocess the labels to convert them to an appropriate format
# Example: [(image1, [x1, y1, x2, y2]), (image2, [x1, y1, x2, y2]), ...]
# Modify batch size, epochs, and other training settings as needed

In [9]:
batch_size = 32  # Adjust as needed
epochs = 50  # Adjust as needed


def preprocess_image(image, bbox):
    if np.shape(image) != (416, 416, 3) or np.shape(bbox) != (4,):
        print(np.shape(image))
        print(np.shape(bbox))
    return image, bbox


# Apply the preprocessing function to the dataset
# dataset_keras = [tuple(preprocess_image(image, bbox)) for image, bbox in dataset]

# Create train and validation dataset
train_dataset = tf.data.Dataset.from_tensor_slices(
    (train_images[:10], train_labels[:10])
)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(5)

val_dataset = tf.data.Dataset.from_tensor_slices((train_images[:10], train_labels[:10]))
val_dataset = val_dataset.batch(batch_size)
val_dataset = val_dataset.shuffle(5)

In [10]:
# Training the model
ssd_model.fit(
    train_dataset,  # Training labels (bounding box coordinates)
    batch_size=batch_size,
    epochs=epochs,
    validation_data=val_dataset,
)

Epoch 1/50
1/1 [==============================] - 98s 98s/step - loss: 23090.4668 - mae: 136.6620 - mse: 23090.4668 - val_loss: 22955.1250 - val_mae: 136.1750 - val_mse: 22955.1250
Epoch 2/50
1/1 [==============================] - 84s 84s/step - loss: 22955.1250 - mae: 136.1750 - mse: 22955.1250 - val_loss: 22955.1250 - val_mae: 136.1750 - val_mse: 22955.1250
Epoch 3/50
1/1 [==============================] - 64s 64s/step - loss: 22955.1250 - mae: 136.1750 - mse: 22955.1250 - val_loss: 22955.1250 - val_mae: 136.1750 - val_mse: 22955.1250
Epoch 4/50
1/1 [==============================] - 55s 55s/step - loss: 22955.1250 - mae: 136.1750 - mse: 22955.1250 - val_loss: 22955.1250 - val_mae: 136.1750 - val_mse: 22955.1250
Epoch 5/50
